## Experimenting with Python Asyncio package

### References
* https://realpython.com/async-io-python/
* https://docs.python.org/3/library/asyncio.html
* https://vibora.io/


In [1]:
import asyncio
import time

### First ever Async program

In [39]:
async def count():
    print("One")
    await asyncio.sleep(1)
    print("Two")

async def main():
    await asyncio.gather(count(), count(), count())


In [50]:
s = time.perf_counter()
# In a standalone program you would start the event loop but this is not needed in a Jupyter notebook
# asyncio.run(main())  
await main()
elapsed = time.perf_counter() - s
print("Executed in {elapsed:0.2f} seconds.".format(elapsed=elapsed))

One
One
One
Two
Two
Two
Executed in 1.00 seconds.


In [46]:
import random

async def count2(coRoutineID):
    print("{id} - One".format(id=coRoutineID))
    randomWait = random.randint(1,10)
    await asyncio.sleep(randomWait)
    print("{id} - Two".format(id=coRoutineID))

    

In [47]:
async def main2():
    funcList = []
    for i in range(10):
        funcList.append(count2(i))
    await asyncio.gather(*funcList)


In [60]:
s = time.perf_counter()
await main2()
elapsed = time.perf_counter() - s
print("Executed in {elapsed:0.2f} seconds.".format(elapsed=elapsed))

0 - One
1 - One
2 - One
3 - One
4 - One
5 - One
6 - One
7 - One
8 - One
9 - One
0 - Two
2 - Two
3 - Two
5 - Two
7 - Two
9 - Two
6 - Two
1 - Two
8 - Two
4 - Two
Executed in 8.00 seconds.


### Awaiting other things
What other things can we wait for??
I actually want to wait for an event to happen and there are things in the asyncio library to do this
BUT first lets just experiment a bit.

So you can wait for anything by creating a coRoutine that does the checking

In [55]:
async def asyncRoutine(id):
    for i in range(10):
        await print("{id} - count {i}".format(id=id, i=i))

In [56]:
await asyncRoutine(1)

1 - count 0


TypeError: object NoneType can't be used in 'await' expression

In [58]:
async def printIt(id,i):
    print("{id} - count {i}".format(id=id, i=i))
    
async def asyncRoutine(id):
    for i in range(10):
        await printIt(id,i)

In [61]:
await asyncRoutine(1)

1 - count 0
1 - count 1
1 - count 2
1 - count 3
1 - count 4
1 - count 5
1 - count 6
1 - count 7
1 - count 8
1 - count 9


#### Something close to what I need

In [7]:
async def stopItNow(eventsDict):
    print("stopItNow called")
    await asyncio.sleep(1)
    eventsDict['MyEvent'] = "STOP"

# Having giveway just pass doesn't work.  stopItNow never gets a look it then   
async def giveway():
    # await asyncio.sleep(1)
    return True

async def checkIt(eventsDict):
    count = 0
    while eventsDict['MyEvent'] is None:
        count += 1
        print("checkIt called for {count} time".format(count=count))
        await giveway()
        
        

In [6]:
eventsDict = {}
eventsDict['MyEvent'] = None

In [ ]:
# WARNING: IF you run this it will not stop and you will have to restart the kernel
# Not too sure why it won;t stop yet?? It appear that without any pause checkIt runs forever
await asyncio.gather(stopItNow(eventsDict), checkIt(eventsDict))

### Asyncio Events

In [4]:
async def waiter(event):
    print('waiting for it ...')
    await event.wait()
    print('... got it!')

async def main():
    # Create an Event object.
    event = asyncio.Event()

    print("Spawn task")
    # Spawn a Task to wait until 'event' is set.
    waiter_task = asyncio.create_task(waiter(event))

    print("Sleep before triggering event")
    # Sleep for 1 second and set the event.
    await asyncio.sleep(1)
    event.set()

    print("Wait for task to finish")
    # Wait until the waiter task is finished.
    await waiter_task



In [5]:
await main()

Spawn task
Sleep before triggering event
waiting for it ...
Wait for task to finish
... got it!


#### Something close to what I need for SyncEndpoint|

In [7]:
# dictionary of asyncio.Event() key'ed by unique responseID
responseEvents = {}
responseDetails = {
    "ID121324242": "This is the response back"
}

In [4]:
def getUniqueResponseID():
    # dummy this for now
    return "ID121324242"

def callAsyncMPOAPI(id):
    # call MPO API pass in responseID
    # MPO process when finished will callback to the webserver on a specific endpoint pass this responseID back
    pass

async def webServerRequest():
    # will call asynchronous MPO process passing it a responseID
    responseID = getUniqueResponseID()
    responseEvents[responseID] = asyncio.Event()
    print('In webserver waiting for response ...')
    callAsyncMPOAPI(responseID)
    await responseEvents[responseID].wait()
    print(responseDetails[responseID])

In [28]:
webserver_task = asyncio.create_task(webServerRequest())
print("Sleep before we send event")
await asyncio.sleep(10)
print("send event")
responseEvents["ID121324242"].set()
# You don't really need the next line (in this case)
await webserver_task

Sleep before we send event
In webserver waiting for response ...
send event
This is the response back


I think the above is basically what I need to implement my SyncEndPoint in https://vibora.io/
Will test this out another day

##  Vibora Tests

Doesn't run from the notebook. Run outside.

In [8]:
from vibora import Vibora, JsonResponse

app = Vibora()

@app.route('/')
async def home():
    responseID = getUniqueResponseID()
    responseEvents[responseID] = asyncio.Event()
    print('In webserver waiting for response ...')
    await responseEvents[responseID].wait()
    return JsonResponse({'hello': 'world'})

@app.route('/response')
async def responseReceiver():
    responseID = getUniqueResponseID()
    responseEvents[responseID].set()
    return JsonResponse({'msg': 'ok'})
    

# Vibora (0.0.7) # http://0.0.0.0:8000


In [7]:
# Once you run this cell you will not be able to do anything else until you stop
app.run(host="0.0.0.0", port=8000)

# Vibora (0.0.7) # http://0.0.0.0:8000


In [11]:
print("hhggh")

hhggh
